In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"

from transformers import AutoTokenizer, AutoModelForCausalLM
from lib import Datasets
from datasets import load_dataset
import copy
from transformers import Trainer, TrainingArguments
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence, List, Literal

import torch
import transformers
from transformers import Trainer
from datasets import load_dataset

2025-02-14 21:27:46.998362: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-14 21:27:47.012345: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739557667.029622 3582209 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739557667.034811 3582209 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-14 21:27:47.052867: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
IGNORE_INDEX = -100

# PROMPT = (
#         "Below is an instruction that describes a task. "
#         "Write a response that appropriately completes the request.\n\n"
#         "### Instruction:\n{instruction}\n\n### Response:"
#     )

PROMPT = (
        "### Task:\n{instruction}\n\n### Solution:"
    )

In [3]:
# model_checkpoint = "distilgpt2"
device = 'cuda:0'

In [4]:
from peft import LoraConfig, TaskType, get_peft_model

peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'up_proj', 'down_proj'])





name = "meta-llama/Llama-2-7b-hf"
# name = "distilgpt2"

tokenizer = AutoTokenizer.from_pretrained(name)
model = AutoModelForCausalLM.from_pretrained(name, torch_dtype=torch.float16, device_map='balanced')

# model = get_peft_model(model, peft_config)

print(model)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_e

In [5]:
# --data_path meta-math/MetaMathQA \
# --dataset_field query response \
    
# python -u train_model.py \
#     --model_name_or_path $BASE_MODEL \
#     --output_dir $OUTPUT \
#     --corda_mode False \
#     --lora_r 128 \
#     --data_path meta-math/MetaMathQA \
#     --dataset_split "train[:100000]" \
#     --dataset_field query response \
#     --num_train_epochs 1 \
#     --per_device_train_batch_size 1 \
#     --gradient_accumulation_steps 128 \
#     --save_strategy "steps" \
#     --save_steps 100 \
#     --save_total_limit 1 \
#     --learning_rate 2e-5 \
#     --weight_decay 0. \
#     --warmup_ratio 0.03 \
#     --lr_scheduler_type "cosine" \
#     --logging_steps 1 \
#     --bf16 True \
#     --tf32 True \
#     --report_to none

In [6]:
def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )


In [7]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        # print(len(label), source_len)
        label[:source_len] = IGNORE_INDEX
        
    return dict(input_ids=input_ids, labels=labels)

def train_tokenize_function(examples, tokenizer, query, response):
    sources = [PROMPT.format_map(dict(instruction=instruction)) for instruction in examples[query]]
    targets = [f"{output}{tokenizer.eos_token}" for output in examples[response]]
    data_dict = preprocess(sources, targets, tokenizer)
    return data_dict

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
        name,
        model_max_length=512,
        padding_side="right",
        use_fast=True,
        trust_remote_code=True,
    )
tokenizer.pad_token_id = tokenizer.eos_token_id

# tokenizer.add_special_tokens({'pad_token': '<pad>'}) #HERE
# model.resize_token_embeddings(len(tokenizer)) #HERE

raw_train_datasets = load_dataset("meta-math/MetaMathQA", split='train[:100]')

train_dataset = raw_train_datasets.map(
        train_tokenize_function,
        batched=True,
        batch_size=3000,
        num_proc=16, # 32
        remove_columns=raw_train_datasets.column_names,
        load_from_cache_file=True,
        desc="Running tokenizer on train dataset",
        fn_kwargs={"tokenizer": tokenizer, "query": "query", "response": "response"}
    )
    

In [9]:
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = [torch.tensor(x) for x in input_ids]
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = [torch.tensor(x) for x in labels]
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [10]:
training_args = TrainingArguments(
    output_dir='./logs/abrakkk',          
    num_train_epochs=1,                   
    per_device_train_batch_size=1,       
    per_device_eval_batch_size=1,       
    gradient_accumulation_steps=1,   
    learning_rate=2e-5,               
    weight_decay=0.0,                   
    warmup_ratio=0.03,                   
    lr_scheduler_type="cosine",          
    logging_steps=1,                       
    fp16=True,
    report_to="tensorboard",              
    logging_dir='./logs/abra',                 
    # use_cpu=True,
)

In [11]:
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)
data_module = dict(train_dataset=train_dataset, data_collator=data_collator)
model.config.use_cache = False

In [12]:
from transformers import Trainer, TrainingArguments, EvalPrediction
from sklearn.metrics import accuracy_score
import numpy as np
from evaluate import load


metric = load('accuracy')

import numpy as np
from evaluate import load

# Load the accuracy metric
metric = load('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    
    predictions = np.argmax(predictions, axis=2).flatten()
    
    labels = labels.flatten()
    
    mask = labels != -100
    
    return metric.compute(predictions=predictions[mask], references=labels[mask])


In [13]:
# trainer = Trainer(model=model, tokenizer=tokenizer, args=script_args, **data_module)

# trainer = Trainer(model=model, args=training_args, **data_module)
trainer = Trainer(model=model, args=training_args, compute_metrics=compute_metrics, **data_module)

In [14]:
print(data_module)

{'train_dataset': Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 100
}), 'data_collator': DataCollatorForSupervisedDataset(tokenizer=LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-hf', vocab_size=32000, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
))}


In [15]:
import math
import time

# model.eval()
# model.config.use_cache = True

# trainer.train()

eval_results = trainer.evaluate(eval_dataset=trainer.train_dataset)
print(eval_results)

../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [0,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [1,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [2,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [3,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [4,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_forward_reduce_cuda_kernel_2d: block: [0,0,0], thread: [5,0,0] Assertion `t >= 0 && t < n_classes` failed.
../aten/src/ATen/native/cuda/Loss.cu:250: nll_loss_f

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
print(data_module['train_dataset']['input_ids'])

[[1, 835, 9330, 29901, 13, 29954, 945, 347, 322, 11131, 526, 23906, 3694, 373, 278, 4280, 10694, 29889, 11131, 3060, 15806, 278, 1298, 395, 29896, 29974, 29906, 29875, 1504, 402, 945, 347, 3060, 15806, 15727, 29896, 29974, 29875, 1504, 1128, 2215, 12435, 526, 402, 945, 347, 322, 11131, 29915, 29879, 3291, 29973, 13, 13, 2277, 29937, 24380, 29901, 1576, 5418, 1546, 1023, 3291, 2427, 29916, 29918, 29896, 29892, 29891, 29918, 29896, 1262, 322, 2427, 29916, 29918, 29906, 29892, 29891, 29918, 29906, 1262, 297, 278, 4280, 10694, 338, 2183, 491, 278, 7063, 779, 3676, 8001, 29916, 29918, 29906, 29899, 29916, 29918, 29896, 4887, 29906, 17108, 29891, 29918, 29906, 29899, 29891, 29918, 29896, 4887, 29906, 4311, 13, 797, 445, 1206, 29892, 11131, 29915, 29879, 1298, 338, 2427, 29896, 29892, 29906, 1262, 322, 402, 945, 347, 29915, 29879, 1298, 338, 2427, 29899, 29896, 29892, 29896, 4935, 13, 6295, 278, 5418, 1546, 1009, 3291, 338, 779, 3676, 29912, 3552, 29899, 29896, 6817, 29898, 29896, 876, 29985,

In [ ]:
for e in trainer.train_dataset:
    print(len(e['input_ids']))

230
187
304
108
272
392
238
196
266
122
83
380
452
130
110
116
203
198
266
306
134
512
250
175
167
340
126
115
161
97
189
248
162
178
151
145
384
197
190
440
343
171
383
392
162
268
157
268
292
315
184
415
152
199
374
168
258
329
288
262
260
319
260
253
209
454
227
192
277
244
145
291
200
370
104
180
158
210
100
222
189
441
273
136
210
283
146
362
172
512
168
146
209
302
91
196
256
141
257
197
